In [21]:
# Import af nødvendige packages
import quandl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [22]:
""" SETTINGS """
source = 'csv' # quandl eller csv
data_interval = 'monthly' # daily, monthy eller yearly - Hvis Quandl = 'daily'
selected = ['NoDur', 'Durbl', 'Manuf', 'Enrgy', 'HiTec', 'Telcm', 'Shops', 'Hlth', 'Utils', 'Other', 'Mkt'] # Angiv ønskede aktiver fra Quandl eller kolonner fra CSV-ark

# Hvis CVS:
file_name = '10 Industry Portfolios - Average Value Weighted Returns.CSV' # navn og file-exstention på data

# Hvis Quandl
date_range = ['2014-1-1', '2016-12-31'] # dato fra og med - dato til og med, format YYYY-MM-DD

In [23]:
# Træk af data fra Quandl
if source == 'quandl':
    quandl.ApiConfig.api_key = "yTPaspmH6wqs9rAdSdmk"
    data = quandl.get_table('WIKI/PRICES', ticker = selected, qopts = {'columns': ['date', 'ticker', 'adj_close']},
                            date = {'gte': date_range[0], 'lte': date_range[1]}, paginate=True)
    clean = data.set_index('date')
    table = clean.pivot(columns='ticker')
    returns_data = table.pct_change()
    
# Træk af data fra CSV-fil (allerede procent-vis ændring)
elif source == 'csv':
    rf = pd.read_csv(file_name, delimiter = ',', header = 0, usecols = ['RF'])
    data = pd.read_csv(file_name, delimiter = ',', header = 0, index_col = 'date', usecols = ['date'] + selected)
    returns_data = np.subtract(data,rf)

returns_data.head() # Eksempel på den procentvise ændring

,NoDur,Durbl,Manuf,Enrgy,HiTec,Telcm,Shops,Hlth,Utils,Other,Mkt
date,,,,,,,,,,,
196307,-0.74,-0.49,-1.68,2.02,-0.96,-0.50,-1.30,0.29,0.53,-1.88,-0.39
196308,4.62,6.30,5.95,3.68,4.89,4.04,6.15,9.31,3.95,5.24,5.07
196309,-1.95,-0.51,-1.03,-3.91,-0.14,2.09,0.69,-4.33,-2.77,-3.43,-1.57
196310,2.37,9.43,2.29,-0.61,8.00,3.11,0.19,3.09,-0.96,1.09,2.53
196311,-1.40,-5.11,0.03,-1.42,-0.56,3.89,-1.50,-1.92,-1.29,-0.04,-0.85


In [24]:
if data_interval == 'monthly': # "1/N opgaven" tager udgangspunkt i månedlig data og sharpe
    returns_monthly = returns_data
    returns_mean = returns_monthly.mean()
    
    cov_matrix = returns_monthly.cov()
    
elif data_interval == 'daily': # primært brugt til data fra Quandl giver årlig sharpe
    returns_daily = returns_data
    returns_mean = returns_daily.mean() * 252
    
    cov_matrix = returns_daily.cov() * 252
    
cov_matrix # Covariance-matricen

,NoDur,Durbl,Manuf,Enrgy,HiTec,Telcm,Shops,Hlth,Utils,Other,Mkt
NoDur,20.128555,17.691304,18.127165,11.494697,17.214965,12.110632,20.569122,17.429386,11.212735,19.824564,16.793447
Durbl,17.691304,32.785327,23.079471,13.488432,26.100298,14.726803,24.085924,15.707215,10.414511,23.247448,20.611461
Manuf,18.127165,23.079471,23.713061,15.245100,25.098295,13.240033,22.221860,17.821404,10.315096,22.813869,20.359101
Enrgy,11.494697,13.488432,15.245100,27.016802,15.088648,9.067012,12.392475,11.573574,11.832497,16.247614,15.260946
HiTec,17.214965,26.100298,25.098295,15.088648,46.084861,18.501968,25.631473,20.877948,7.591553,24.887739,25.909885
Telcm,12.110632,14.726803,13.240033,9.067012,18.501968,21.761713,15.353761,12.021368,9.027744,15.419854,14.942959
Shops,20.569122,24.085924,22.221860,12.392475,25.631473,15.353761,29.473522,18.492237,10.391593,23.787075,21.025477
Hlth,17.429386,15.707215,17.821404,11.573574,20.877948,12.021368,18.492237,25.948375,9.577781,19.081650,17.474101
Utils,11.212735,10.414511,10.315096,11.832497,7.591553,9.027744,10.391593,9.577781,16.675613,13.065021,10.598270
Other,19.824564,23.247448,22.813869,16.247614,24.887739,15.419854,23.787075,19.081650,13.065021,27.350367,21.722339


In [25]:
port_returns = []
port_variance = []
sharpe_ratio = []
stock_weights = []

In [26]:
# simulerer 1/N portfølje fra 1963-07 til 2004-11 som rollover med 60 måneders interval:
for month in range(0, len(returns_data) - 60):
    
    # Danner nyt data interval, returns og cov-matrix for hvert interval
    new_returns_data = returns_data[month:month + 60]
    returns_monthly = new_returns_data
    returns_mean = returns_monthly.mean()
    cov_matrix = returns_monthly.cov()
      
    # udregner profit, volatilitet og sharpe
    tl = np.dot(np.linalg.inv(cov_matrix), returns_mean)
    nl = np.dot(np.ones(11), np.dot(np.linalg.inv(cov_matrix), returns_mean))
    weights = tl / nl
    
    # udregner afkast for måned T + 1 med de udregnede optimale vægte
    returns = np.dot(weights, returns_data[month + 60:month + 61].mean())
    variance = np.dot(weights, np.dot(cov_matrix, weights))
    sharpe = returns / np.sqrt(variance)

    # indsætter overstående udregninger i vektorerne
    sharpe_ratio.append(sharpe)
    port_returns.append(returns)
    port_variance.append(variance)
    stock_weights.append(weights)

In [27]:
# definerer portfolier som en sammensætning af overstående udregninger
portfolio = {'Returns': port_returns,
             'Variance': port_variance,
             'Sharpe Ratio': sharpe_ratio}

In [28]:
# definerer de enkelte vægte af de valgte aktiver
for i in range(len((selected))):
    symbol = selected[i]
    portfolio[symbol+' Weight'] = [Weight[i] for Weight in stock_weights]

In [29]:
# sætter vores portføljer som dataframe i pandas
df = pd.DataFrame(portfolio)
column_order = ['Returns', 'Variance', 'Sharpe Ratio'] + [symbol+' Weight' for symbol in selected]
df = df[column_order]

df.head() # Eksempel på de tilfældigt genereret porteføljer

,Returns,Variance,Sharpe Ratio,NoDur Weight,Durbl Weight,Manuf Weight,Enrgy Weight,HiTec Weight,Telcm Weight,Shops Weight,Hlth Weight,Utils Weight,Other Weight,Mkt Weight
0,6.210466,30.875909,1.117672,2.676378,1.544866,5.715830,5.198498,3.140763,1.370313,1.903290,1.511928,1.547120,1.626682,-25.235667
1,2.197234,32.453110,0.385699,2.720000,1.815273,6.495033,5.708616,3.360725,1.622802,2.067058,1.555690,1.704892,1.938046,-27.988136
2,1.111550,33.274263,0.192697,3.042496,1.782807,6.725192,5.848831,3.326696,1.646228,2.031424,1.446008,1.580043,1.926381,-28.356106
3,0.994201,31.463105,0.177245,2.856587,1.580860,5.506478,5.476601,2.907453,1.379123,1.819827,1.345229,1.238315,2.094594,-25.205067
4,6.924722,32.726204,1.210471,2.786230,1.404639,6.185166,5.849048,2.609267,1.397605,2.129683,1.466918,1.165981,2.172076,-26.166612


In [30]:
df['Returns'].mean() / df['Returns'].std()

0.0586012943590723